In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import normaltest
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

import xgboost as xgb
from sklearn.svm import SVR
from pyearth import Earth
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold

from catboost import CatBoostRegressor, Pool

import lightgbm as lgb

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [17]:
train = pd.read_csv('train.csv')
test =pd.read_csv('test.csv')
test_ids = test['id']
test.drop(columns = 'id',inplace = True)
X = train.drop(columns = ['y','id'])
y = train['y']
cols = X.columns
X = X.astype(float)
y = y.astype(float)
test = test.astype(float)

y = np.log(y)

counts = pd.DataFrame(X.nunique()).reset_index()
todrop = counts[counts[0] == 1]['index']

X.drop(columns = todrop,inplace = True)
test.drop(columns = todrop, inplace = True)

sc = StandardScaler()
sc.fit(X)
X = pd.DataFrame(sc.transform(X))
test = pd.DataFrame(sc.transform(test))

Xknn = KNNImputer(n_neighbors=7)
Xknn.fit(X)
X = pd.DataFrame(Xknn.transform(X))
test = pd.DataFrame(Xknn.transform(test))

In [29]:
X.isna().sum().max()

0

In [30]:
X = add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

for i in range(len(X.columns)):
    vif_data.loc[i,'VIF'] = variance_inflation_factor(X.values, i)

print(vif_data)

C:\Users\Arielle\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\Arielle\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Arielle\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


    feature           VIF
0     const  0.000000e+00
1         0  1.334586e+05
2         1  1.464288e+05
3         2  9.264701e+01
4         3  6.100465e+01
..      ...           ...
750     749           inf
751     750           inf
752     751  1.594674e+07
753     752  1.772276e+06
754     753  6.468168e+00

[755 rows x 2 columns]


In [47]:
vif_keep = list(vif_data[(vif_data['VIF']<3) & (vif_data['feature'] != 'const')]['feature'])

X = X[vif_keep]
test = test[vif_keep]

In [56]:
catmodel = CatBoostRegressor().fit(X,y)
xgbmodel = xgb.XGBRegressor().fit(X,y)

preds = catmodel.predict(test)*0.8 + xgbmodel.predict(test)*0.2

Learning rate set to 0.053413
0:	learn: 0.9884857	total: 161ms	remaining: 2m 40s
1:	learn: 0.9794373	total: 164ms	remaining: 1m 21s
2:	learn: 0.9714858	total: 168ms	remaining: 55.7s
3:	learn: 0.9645765	total: 170ms	remaining: 42.4s
4:	learn: 0.9589017	total: 173ms	remaining: 34.4s
5:	learn: 0.9532839	total: 176ms	remaining: 29.2s
6:	learn: 0.9470832	total: 180ms	remaining: 25.5s
7:	learn: 0.9415509	total: 183ms	remaining: 22.7s
8:	learn: 0.9364708	total: 186ms	remaining: 20.5s
9:	learn: 0.9314996	total: 189ms	remaining: 18.7s
10:	learn: 0.9274963	total: 192ms	remaining: 17.2s
11:	learn: 0.9235727	total: 196ms	remaining: 16.1s
12:	learn: 0.9197368	total: 198ms	remaining: 15s
13:	learn: 0.9163891	total: 201ms	remaining: 14.2s
14:	learn: 0.9130758	total: 205ms	remaining: 13.5s
15:	learn: 0.9098576	total: 208ms	remaining: 12.8s
16:	learn: 0.9071635	total: 212ms	remaining: 12.3s
17:	learn: 0.9045490	total: 216ms	remaining: 11.8s
18:	learn: 0.9020098	total: 219ms	remaining: 11.3s
19:	learn: 

163:	learn: 0.7965991	total: 529ms	remaining: 2.7s
164:	learn: 0.7957371	total: 531ms	remaining: 2.69s
165:	learn: 0.7950333	total: 533ms	remaining: 2.68s
166:	learn: 0.7943973	total: 536ms	remaining: 2.67s
167:	learn: 0.7938271	total: 538ms	remaining: 2.66s
168:	learn: 0.7934574	total: 540ms	remaining: 2.65s
169:	learn: 0.7928641	total: 542ms	remaining: 2.64s
170:	learn: 0.7924159	total: 544ms	remaining: 2.63s
171:	learn: 0.7920295	total: 546ms	remaining: 2.63s
172:	learn: 0.7916763	total: 548ms	remaining: 2.62s
173:	learn: 0.7910946	total: 550ms	remaining: 2.61s
174:	learn: 0.7906154	total: 552ms	remaining: 2.6s
175:	learn: 0.7903860	total: 554ms	remaining: 2.59s
176:	learn: 0.7900107	total: 556ms	remaining: 2.58s
177:	learn: 0.7893939	total: 558ms	remaining: 2.58s
178:	learn: 0.7887394	total: 560ms	remaining: 2.57s
179:	learn: 0.7880792	total: 562ms	remaining: 2.56s
180:	learn: 0.7877479	total: 564ms	remaining: 2.55s
181:	learn: 0.7873931	total: 566ms	remaining: 2.54s
182:	learn: 0.

344:	learn: 0.7065603	total: 888ms	remaining: 1.69s
345:	learn: 0.7062889	total: 890ms	remaining: 1.68s
346:	learn: 0.7058338	total: 892ms	remaining: 1.68s
347:	learn: 0.7054147	total: 894ms	remaining: 1.67s
348:	learn: 0.7049380	total: 896ms	remaining: 1.67s
349:	learn: 0.7045678	total: 898ms	remaining: 1.67s
350:	learn: 0.7042331	total: 900ms	remaining: 1.66s
351:	learn: 0.7037779	total: 902ms	remaining: 1.66s
352:	learn: 0.7034291	total: 904ms	remaining: 1.66s
353:	learn: 0.7030399	total: 906ms	remaining: 1.65s
354:	learn: 0.7024862	total: 909ms	remaining: 1.65s
355:	learn: 0.7020405	total: 910ms	remaining: 1.65s
356:	learn: 0.7017737	total: 912ms	remaining: 1.64s
357:	learn: 0.7013122	total: 915ms	remaining: 1.64s
358:	learn: 0.7008894	total: 917ms	remaining: 1.64s
359:	learn: 0.7003703	total: 918ms	remaining: 1.63s
360:	learn: 0.7000245	total: 920ms	remaining: 1.63s
361:	learn: 0.6993746	total: 923ms	remaining: 1.63s
362:	learn: 0.6990014	total: 924ms	remaining: 1.62s
363:	learn: 

525:	learn: 0.6359591	total: 1.24s	remaining: 1.12s
526:	learn: 0.6355759	total: 1.25s	remaining: 1.12s
527:	learn: 0.6351319	total: 1.25s	remaining: 1.12s
528:	learn: 0.6347228	total: 1.25s	remaining: 1.11s
529:	learn: 0.6342389	total: 1.25s	remaining: 1.11s
530:	learn: 0.6339657	total: 1.25s	remaining: 1.11s
531:	learn: 0.6337009	total: 1.26s	remaining: 1.1s
532:	learn: 0.6333338	total: 1.26s	remaining: 1.1s
533:	learn: 0.6330257	total: 1.26s	remaining: 1.1s
534:	learn: 0.6325908	total: 1.26s	remaining: 1.1s
535:	learn: 0.6324003	total: 1.26s	remaining: 1.09s
536:	learn: 0.6320620	total: 1.27s	remaining: 1.09s
537:	learn: 0.6318006	total: 1.27s	remaining: 1.09s
538:	learn: 0.6312531	total: 1.27s	remaining: 1.09s
539:	learn: 0.6310321	total: 1.27s	remaining: 1.08s
540:	learn: 0.6307050	total: 1.27s	remaining: 1.08s
541:	learn: 0.6304861	total: 1.28s	remaining: 1.08s
542:	learn: 0.6301441	total: 1.28s	remaining: 1.08s
543:	learn: 0.6299506	total: 1.28s	remaining: 1.07s
544:	learn: 0.62

703:	learn: 0.5754604	total: 1.6s	remaining: 674ms
704:	learn: 0.5751576	total: 1.6s	remaining: 671ms
705:	learn: 0.5749678	total: 1.61s	remaining: 669ms
706:	learn: 0.5745561	total: 1.61s	remaining: 667ms
707:	learn: 0.5742276	total: 1.61s	remaining: 665ms
708:	learn: 0.5739170	total: 1.61s	remaining: 662ms
709:	learn: 0.5736331	total: 1.62s	remaining: 660ms
710:	learn: 0.5734811	total: 1.62s	remaining: 658ms
711:	learn: 0.5731474	total: 1.62s	remaining: 656ms
712:	learn: 0.5727338	total: 1.62s	remaining: 653ms
713:	learn: 0.5722052	total: 1.62s	remaining: 651ms
714:	learn: 0.5718896	total: 1.63s	remaining: 648ms
715:	learn: 0.5714119	total: 1.63s	remaining: 646ms
716:	learn: 0.5709262	total: 1.63s	remaining: 644ms
717:	learn: 0.5705610	total: 1.63s	remaining: 641ms
718:	learn: 0.5703194	total: 1.64s	remaining: 639ms
719:	learn: 0.5699894	total: 1.64s	remaining: 637ms
720:	learn: 0.5694979	total: 1.64s	remaining: 634ms
721:	learn: 0.5691841	total: 1.64s	remaining: 632ms
722:	learn: 0.

885:	learn: 0.5213484	total: 1.96s	remaining: 253ms
886:	learn: 0.5210746	total: 1.96s	remaining: 250ms
887:	learn: 0.5207982	total: 1.97s	remaining: 248ms
888:	learn: 0.5207188	total: 1.97s	remaining: 246ms
889:	learn: 0.5204261	total: 1.97s	remaining: 244ms
890:	learn: 0.5201853	total: 1.97s	remaining: 242ms
891:	learn: 0.5199813	total: 1.98s	remaining: 239ms
892:	learn: 0.5196580	total: 1.98s	remaining: 237ms
893:	learn: 0.5193948	total: 1.98s	remaining: 235ms
894:	learn: 0.5190096	total: 1.98s	remaining: 233ms
895:	learn: 0.5186611	total: 1.98s	remaining: 230ms
896:	learn: 0.5185036	total: 1.99s	remaining: 228ms
897:	learn: 0.5182413	total: 1.99s	remaining: 226ms
898:	learn: 0.5179569	total: 1.99s	remaining: 224ms
899:	learn: 0.5176182	total: 1.99s	remaining: 221ms
900:	learn: 0.5174498	total: 1.99s	remaining: 219ms
901:	learn: 0.5171105	total: 2s	remaining: 217ms
902:	learn: 0.5167417	total: 2s	remaining: 215ms
903:	learn: 0.5164334	total: 2s	remaining: 212ms
904:	learn: 0.5160855

In [62]:
preds = np.exp(preds)

In [64]:
predictions = pd.DataFrame({'id':test_ids,'y':preds})
predictions.head()

,id,y
0,5380,4.344145
1,5381,12.816538
2,5382,3.211475
3,5383,4.917277
4,5384,3.460019


In [65]:
predictions.to_csv('cattest.csv',index=False) #:D